In [1]:
!pip install pandas

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
#from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [3]:
heart_dataset=pd.read_csv("heart.csv")
heart_dataset

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,59,1,1,140,221,0,1,164,1,0.0,2,0,2,1
1021,60,1,0,125,258,0,0,141,1,2.8,1,1,3,0
1022,47,1,0,110,275,0,0,118,1,1.0,1,1,2,0
1023,50,0,0,110,254,0,0,159,0,0.0,2,0,2,1


In [4]:
heart_dataset.shape

(1025, 14)

In [5]:
heart_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1025 non-null   int64  
 1   sex       1025 non-null   int64  
 2   cp        1025 non-null   int64  
 3   trestbps  1025 non-null   int64  
 4   chol      1025 non-null   int64  
 5   fbs       1025 non-null   int64  
 6   restecg   1025 non-null   int64  
 7   thalach   1025 non-null   int64  
 8   exang     1025 non-null   int64  
 9   oldpeak   1025 non-null   float64
 10  slope     1025 non-null   int64  
 11  ca        1025 non-null   int64  
 12  thal      1025 non-null   int64  
 13  target    1025 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 112.2 KB


In [6]:
heart_dataset.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [7]:
heart_dataset.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,1025.000000,1025.000000,1025.000000,1025.000000,1025.00000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000
mean,54.434146,0.695610,0.942439,131.611707,246.00000,0.149268,0.529756,149.114146,0.336585,1.071512,1.385366,0.754146,2.323902,0.513171
std,9.072290,0.460373,1.029641,17.516718,51.59251,0.356527,0.527878,23.005724,0.472772,1.175053,0.617755,1.030798,0.620660,0.500070
min,29.000000,0.000000,0.000000,94.000000,126.00000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,48.000000,0.000000,0.000000,120.000000,211.00000,0.000000,0.000000,132.000000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,56.000000,1.000000,1.000000,130.000000,240.00000,0.000000,1.000000,152.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,275.00000,0.000000,1.000000,166.000000,1.000000,1.800000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.00000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [8]:
heart_dataset['target'].value_counts()

target
1    526
0    499
Name: count, dtype: int64

In [9]:
# Data processing
cate_val=[]
cont_val=[]

for column in heart_dataset.columns:
    if heart_dataset[column].nunique() <=10:
        cate_val.append(column)
    else:
        cont_val.append(column)

In [10]:
cate_val

['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal', 'target']

In [11]:
cont_val

['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

In [12]:
heart_dataset['cp'].unique()

array([0, 1, 2, 3], dtype=int64)

In [13]:
from sklearn.preprocessing import StandardScaler
st=StandardScaler()
heart_dataset[cont_val]=st.fit_transform(heart_dataset[cont_val])

In [14]:
heart_dataset

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,-0.268437,1,0,-0.377636,-0.659332,0,1,0.821321,0,-0.060888,2,2,3,0
1,-0.158157,1,0,0.479107,-0.833861,1,0,0.255968,1,1.727137,0,0,3,0
2,1.716595,1,0,0.764688,-1.396233,0,1,-1.048692,1,1.301417,0,0,3,0
3,0.724079,1,0,0.936037,-0.833861,0,1,0.516900,0,-0.912329,2,1,3,0
4,0.834359,0,0,0.364875,0.930822,1,1,-1.874977,0,0.705408,1,3,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,0.503520,1,1,0.479107,-0.484803,0,1,0.647366,1,-0.912329,2,0,2,1
1021,0.613800,1,0,-0.377636,0.232705,0,0,-0.352873,1,1.471705,1,1,3,0
1022,-0.819834,1,0,-1.234378,0.562371,0,0,-1.353113,1,-0.060888,1,1,2,0
1023,-0.488996,0,0,-1.234378,0.155137,0,0,0.429923,0,-0.912329,2,0,2,1


In [15]:
X= heart_dataset.drop('target',axis=1)
y=heart_dataset['target']

In [16]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=5)

In [17]:
X_train

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
390,0.393241,0,1,0.250643,1.415625,1,0,0.125502,0,-0.912329,2,2,2
864,0.503520,1,3,1.621431,0.523587,0,0,-1.048692,0,-0.912329,2,0,2
489,0.724079,1,2,1.050269,-0.058176,1,1,-0.526828,1,-0.060888,1,0,2
165,0.503520,1,0,2.192593,1.551370,0,0,-0.396362,1,1.982569,0,0,3
40,1.165198,0,2,1.621431,2.210702,0,0,0.082013,0,-0.231177,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
998,-1.371232,1,0,0.250643,1.338056,0,1,-1.048692,1,0.620264,1,0,1
701,-2.143189,1,0,-0.663216,-0.930822,0,1,-0.831249,1,0.449976,1,0,3
1007,0.172682,1,3,-0.663216,-1.027782,0,0,0.560389,0,0.705408,1,0,3
206,0.503520,1,2,-0.320519,-0.542979,1,1,-0.657294,0,0.960840,1,1,1


In [18]:
y_train

390     0
864     0
489     1
165     0
40      1
       ..
998     0
701     0
1007    1
206     0
867     1
Name: target, Length: 820, dtype: int64

In [19]:
X_test

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
293,0.503520,1,2,1.050269,-0.659332,1,1,0.342945,0,0.449976,2,0,2
697,1.385757,1,2,1.164502,-0.659332,0,0,0.038525,0,-0.231177,1,0,3
353,0.282961,1,0,-1.234378,-0.872645,0,1,-1.005203,1,0.364832,1,0,1
481,0.944639,0,0,1.050269,3.122131,0,0,0.212479,0,2.493434,1,3,3
823,-0.047877,1,0,0.479107,-0.135745,0,1,0.473411,0,0.109400,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
384,-2.143189,1,0,-0.320519,0.698116,0,0,0.299457,1,-0.912329,2,0,3
928,-0.047877,1,0,-0.434752,0.387842,0,0,-1.744511,1,0.960840,1,1,3
737,1.385757,1,0,-0.663216,-0.329666,0,0,-0.874737,1,1.301417,1,2,3
505,-1.150673,0,2,-0.777449,-0.077568,0,1,-0.004964,0,-0.656897,1,1,2


In [20]:
y_test

293    1
697    0
353    1
481    0
823    1
      ..
384    0
928    0
737    0
505    1
875    0
Name: target, Length: 205, dtype: int64

Logistic Regression

In [21]:
log=LogisticRegression()
log.fit(X_train,y_train)

LogisticRegression()

In [22]:
y_pred=log.predict(X_test)

In [23]:
accuracy_score(y_test,y_pred)

0.848780487804878

SVC

In [24]:
from sklearn import svm
svm=svm.SVC()
svm.fit(X_train,y_train)

SVC()

In [25]:
y_pred2=svm.predict(X_test)

In [26]:
accuracy_score(y_test,y_pred2)

0.8878048780487805

KNeighbors Classifier

In [27]:
knn=KNeighborsClassifier()
knn.fit(X_train,y_train)

KNeighborsClassifier()

In [28]:
y_pred3=knn.predict(X_test)

In [29]:
accuracy_score(y_test,y_pred3)

0.8634146341463415

In [30]:
score=[]

for k in range(1,40):
    knn=KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train,y_train)
    y_pred4=knn.predict(X_test)
    score.append(accuracy_score(y_test,y_pred4))

In [31]:
score

[0.9853658536585366,
 0.975609756097561,
 0.9609756097560975,
 0.9073170731707317,
 0.8634146341463415,
 0.8878048780487805,
 0.8634146341463415,
 0.8536585365853658,
 0.8780487804878049,
 0.8780487804878049,
 0.8536585365853658,
 0.8536585365853658,
 0.8341463414634146,
 0.8390243902439024,
 0.848780487804878,
 0.8585365853658536,
 0.8585365853658536,
 0.8585365853658536,
 0.8341463414634146,
 0.8292682926829268,
 0.824390243902439,
 0.8292682926829268,
 0.824390243902439,
 0.824390243902439,
 0.8195121951219512,
 0.8195121951219512,
 0.8195121951219512,
 0.824390243902439,
 0.824390243902439,
 0.824390243902439,
 0.8146341463414634,
 0.8146341463414634,
 0.8146341463414634,
 0.8048780487804879,
 0.8048780487804879,
 0.8048780487804879,
 0.8146341463414634,
 0.8097560975609757,
 0.8048780487804879]

In [32]:
knn=KNeighborsClassifier(n_neighbors=2)
knn.fit(X_train,y_train)
y_pred4=knn.predict(X_test)
accuracy_score(y_test,y_pred4)

0.975609756097561

decision tree classifier

In [33]:
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier()
dt.fit(X_train,y_train)

DecisionTreeClassifier()

In [34]:
y_pred5=dt.predict(X_test)
accuracy_score(y_test,y_pred5)

0.9853658536585366

Random Forest Classifier

In [35]:
rf=RandomForestClassifier()
rf.fit(X_train,y_train)

RandomForestClassifier()

In [36]:
y_pred6=rf.predict(X_test)
accuracy_score(y_test,y_pred6)

1.0

In [55]:
final_data=pd.DataFrame({'Models':['LR','SVM','KNN','DT','RF'],'Accuracy':[accuracy_score(y_test,y_pred),
                                                                      accuracy_score(y_test,y_pred2),
                                                                      accuracy_score(y_test,y_pred3),
                                                                      accuracy_score(y_test,y_pred5),
                                                                      accuracy_score(y_test,y_pred6)]})

In [56]:
final_data

,Models,Accuracy
0,LR,0.848780
1,SVM,0.887805
2,KNN,0.863415
3,DT,0.985366
4,RF,1.000000


In [57]:
knn.score(X_test,y_test)

0.975609756097561

In [58]:
knn_predict=knn.predict(X_test)
knn_predict_dict={'predict_values':knn_predict,'original_values':y_test}
#creating new dataframe
op=pd.DataFrame(knn_predict_dict)
op[op['predict_values']!=op['original_values']]

,predict_values,original_values
1001,0,1
906,0,1
947,0,1
568,0,1
426,0,1


In [59]:
svm_predict=svm.predict(X_test)
svm_predict_dict={'predict_values':svm_predict,'original_values':y_test}
#creating new dataframe
op1=pd.DataFrame(svm_predict_dict)
op1[op1['predict_values']!=op1['original_values']]

,predict_values,original_values
697,1,0
353,0,1
854,1,0
439,1,0
1012,1,0
1016,1,0
630,1,0
987,1,0
358,1,0
805,0,1


In [64]:
X1= heart_dataset.drop('target',axis=1)
y1= heart_dataset['target']

In [65]:
X1.shape

(1025, 13)

In [71]:
rfc=RandomForestClassifier()
rfc.fit(X1,y1)

RandomForestClassifier()

Prediction for a new dataset

In [75]:
new=pd.DataFrame({
    'age':50,
    'sex':1,
    'cp':0,
    'trestbps':125,	
    'chol':212,
    'fbs':0,
    'restecg':1,
    'thalach':168,
    'exang':0,
    'oldpeak':1.0,
    'slope':2,
    'ca':2,
    'thal':3,   
},index=[0])

In [76]:
new

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,50,1,0,125,212,0,1,168,0,1.0,2,2,3


In [77]:
pred=rfc.predict(new)
if pred[0]==0:
    print("No Heart Disease")
else:
    print("Heart Disease")

No Heart Disease


Saving the model

In [79]:
import joblib

In [80]:
joblib.dump(rfc,'model_joblib_heart')

['model_joblib_heart']

In [81]:
model=joblib.load('model_joblib_heart')

In [82]:
model.predict(new)

array([0], dtype=int64)

In [83]:
from tkinter import *

In [84]:
def show_entry_fields():
    p1=int(e1.get())
    p2=int(e2.get())
    p3=int(e3.get())
    p4=int(e4.get())
    p5=int(e5.get())
    p6=int(e6.get())
    p7=int(e7.get())
    p8=int(e8.get())
    p9=int(e9.get())
    p10=int(e10.get())
    p11=int(e11.get())
    p12=int(e12.get())
    p13=int(e13.get())
    model=joblib.load('model_joblib_heart')
    result=model.predict([[p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13]])

    if result == 0:
        Label(master,text="No Heart Disease").grid(row=31)
    else:
        Label(master,text="Probability of Heart Disease").grid(row=31)


In [91]:
master= Tk()
master.title("Heart Disease Prediction System")

label=Label(master,text="Heart Disease Prediction System",bg="black",fg="white").grid(row=0,columnspan=2)

Label(master, text="Enter your Age").grid(row=1)
Label(master, text="Male or Female[0/1]").grid(row=2)
Label(master, text="Enter value of cp").grid(row=3)
Label(master, text="Enter value of trestbps").grid(row=4)
Label(master, text="Enter value of chol").grid(row=5)
Label(master, text="Enter value of fbs").grid(row=6)
Label(master, text="Enter value of restecg").grid(row=7)
Label(master, text="Enter value of thalach").grid(row=8)
Label(master, text="Enter value of exang").grid(row=9)
Label(master, text="Enter value of oldpeak").grid(row=10)
Label(master, text="Enter value of slope").grid(row=11)
Label(master, text="Enter value of ca").grid(row=12)
Label(master, text="Enter value of thal").grid(row=13)

e1 = Entry(master)
e2 = Entry(master)
e3 = Entry(master)
e4 = Entry(master)
e5 = Entry(master)
e6 = Entry(master)
e7 = Entry(master)
e8 = Entry(master)
e9 = Entry(master)
e10 = Entry(master)
e11 = Entry(master)
e12 = Entry(master)
e13 = Entry(master)

e1.grid(row=1,column=1)
e2.grid(row=2,column=1)
e3.grid(row=3,column=1)
e4.grid(row=4,column=1)
e5.grid(row=5,column=1)
e6.grid(row=6,column=1)
e7.grid(row=7,column=1)
e8.grid(row=8,column=1)
e9.grid(row=9,column=1)
e10.grid(row=10,column=1)
e11.grid(row=11,column=1)
e12.grid(row=12,column=1)
e13.grid(row=13,column=1)

Button(master,text="Predict",command=show_entry_fields).grid()

mainloop()

C:\Users\swapnil\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
